<a href="https://colab.research.google.com/github/binhluong84/Machine-Learning/blob/main/LendingClub_Loan_Grade_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
data = pd.read_csv('/gdrive/MyDrive/All Lending Club loan/accepted_2007_to_2018Q4.csv.gz', compression='gzip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,88985880,NaN,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260697,88224441,NaN,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,...,NaN,NaN,Cash,Y,Mar-2019,ACTIVE,Mar-2019,10000.0,44.82,1.0
2260698,88215728,NaN,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Cutting the data in half to avoid out-of-memory issues

data = data.sample(frac=0.5, axis=0, random_state=42).reset_index(drop=True)

In [ ]:
data.isna().mean().sort_values()

id                                            0.000000
fico_range_high                               0.000017
hardship_flag                                 0.000017
revol_bal                                     0.000017
initial_list_status                           0.000017
                                                ...   
hardship_reason                               0.995193
hardship_dpd                                  0.995193
hardship_loan_status                          0.995193
orig_projected_additional_accrued_interest    0.996196
member_id                                     1.000000
Length: 151, dtype: float64

In [ ]:
data = data.drop(data.loc[:, data.isna().mean().sort_values() > 0.3].columns, axis=1)

In [ ]:
data

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,...,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,N,Cash,N
1,16411620,9600.0,9600.0,9600.0,36 months,12.99,323.42,C,C1,NaN,...,50.0,1.0,0.0,11600.0,4509.0,2400.0,0.0,N,Cash,N
2,45122316,4000.0,4000.0,4000.0,36 months,6.68,122.93,A,A3,System Analyst,...,0.0,0.0,0.0,222616.0,64253.0,5600.0,76154.0,N,Cash,N
3,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,...,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,N,Cash,N
4,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,...,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130345,8581088,12000.0,12000.0,12000.0,60 months,15.61,289.34,C,C3,Loan Officer,...,50.0,0.0,0.0,355441.0,47127.0,41800.0,15241.0,N,Cash,N
1130346,5646193,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,C5,Albert Restaurant Group,...,100.0,0.0,0.0,279063.0,56961.0,3600.0,46901.0,N,Cash,N
1130347,9294954,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,D2,manager,...,0.0,0.0,0.0,149577.0,46878.0,20800.0,45490.0,N,Cash,N
1130348,59249382,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,C5,Custodian,...,0.0,0.0,0.0,169570.0,26843.0,3200.0,0.0,N,Cash,N


In [ ]:
data = data.dropna(axis=0).reset_index(drop=True)

In [ ]:
data

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,...,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,N,Cash,N
1,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,...,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,N,Cash,N
2,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,...,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,N,Cash,N
3,88273234,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,B2,Manager,...,60.0,0.0,0.0,322295.0,33695.0,23900.0,29995.0,N,Cash,N
4,88725013,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,C4,Manager,...,80.0,0.0,0.0,70820.0,54296.0,17300.0,47920.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,78228930,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,C5,police officer,...,66.7,0.0,0.0,137307.0,23860.0,9200.0,15207.0,N,Cash,N
826526,5646193,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,C5,Albert Restaurant Group,...,100.0,0.0,0.0,279063.0,56961.0,3600.0,46901.0,N,Cash,N
826527,9294954,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,D2,manager,...,0.0,0.0,0.0,149577.0,46878.0,20800.0,45490.0,N,Cash,N
826528,59249382,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,C5,Custodian,...,0.0,0.0,0.0,169570.0,26843.0,3200.0,0.0,N,Cash,N


In [ ]:
data.isna().sum().sum()

0

In [ ]:
data

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,39651438,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,...,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,N,Cash,N
1,125356772,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,...,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,N,Cash,N
2,128490686,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,...,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,N,Cash,N
3,88273234,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,B2,Manager,...,60.0,0.0,0.0,322295.0,33695.0,23900.0,29995.0,N,Cash,N
4,88725013,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,C4,Manager,...,80.0,0.0,0.0,70820.0,54296.0,17300.0,47920.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,78228930,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,C5,police officer,...,66.7,0.0,0.0,137307.0,23860.0,9200.0,15207.0,N,Cash,N
826526,5646193,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,C5,Albert Restaurant Group,...,100.0,0.0,0.0,279063.0,56961.0,3600.0,46901.0,N,Cash,N
826527,9294954,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,D2,manager,...,0.0,0.0,0.0,149577.0,46878.0,20800.0,45490.0,N,Cash,N
826528,59249382,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,C5,Custodian,...,0.0,0.0,0.0,169570.0,26843.0,3200.0,0.0,N,Cash,N


In [ ]:
unneeded_columns = ['id', 'sub_grade', 'emp_title', 'url', 'title', 'zip_code']

In [ ]:
unneeded_columns

['id', 'sub_grade', 'emp_title', 'url', 'title', 'zip_code']

In [ ]:
{column: list(data[column].unique()) for column in data.drop(unneeded_columns, axis=1).columns if data.dtypes[column] == 'object'}

{'term': [' 60 months', ' 36 months'],
 'grade': ['B', 'E', 'C', 'A', 'D', 'F', 'G'],
 'emp_length': ['10+ years',
  '5 years',
  '7 years',
  '2 years',
  '1 year',
  '< 1 year',
  '4 years',
  '6 years',
  '3 years',
  '9 years',
  '8 years'],
 'home_ownership': ['MORTGAGE', 'RENT', 'OWN', 'ANY', 'NONE', 'OTHER'],
 'verification_status': ['Verified', 'Not Verified', 'Source Verified'],
 'issue_d': ['Feb-2015',
  'Dec-2017',
  'Feb-2018',
  'Aug-2016',
  'Oct-2016',
  'Jan-2018',
  'Jan-2015',
  'Sep-2014',
  'May-2015',
  'Apr-2016',
  'Sep-2016',
  'Oct-2017',
  'Jun-2015',
  'Jun-2013',
  'Oct-2013',
  'Jul-2017',
  'May-2018',
  'Jan-2013',
  'Jan-2017',
  'Nov-2018',
  'Jul-2014',
  'Sep-2018',
  'Aug-2017',
  'Oct-2015',
  'Jul-2016',
  'Aug-2018',
  'Aug-2015',
  'Sep-2015',
  'Feb-2017',
  'Mar-2017',
  'Mar-2018',
  'Nov-2014',
  'Mar-2013',
  'Oct-2014',
  'Apr-2015',
  'Feb-2014',
  'Nov-2017',
  'Apr-2013',
  'May-2013',
  'Mar-2014',
  'Jul-2018',
  'Dec-2016',
  'Jul-201

In [ ]:
data = data.drop(unneeded_columns, axis=1)

In [ ]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,10+ years,MORTGAGE,120000.0,...,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,N,Cash,N
1,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,10+ years,RENT,52000.0,...,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,N,Cash,N
2,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,10+ years,OWN,65000.0,...,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,N,Cash,N
3,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,10+ years,MORTGAGE,100000.0,...,60.0,0.0,0.0,322295.0,33695.0,23900.0,29995.0,N,Cash,N
4,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,5 years,RENT,120000.0,...,80.0,0.0,0.0,70820.0,54296.0,17300.0,47920.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,10+ years,MORTGAGE,70000.0,...,66.7,0.0,0.0,137307.0,23860.0,9200.0,15207.0,N,Cash,N
826526,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,10+ years,MORTGAGE,98000.0,...,100.0,0.0,0.0,279063.0,56961.0,3600.0,46901.0,N,Cash,N
826527,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,7 years,MORTGAGE,49000.0,...,0.0,0.0,0.0,149577.0,46878.0,20800.0,45490.0,N,Cash,N
826528,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,7 years,MORTGAGE,37000.0,...,0.0,0.0,0.0,169570.0,26843.0,3200.0,0.0,N,Cash,N


In [ ]:
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

In [ ]:
data['issue_d']

0         Feb-2015
1         Dec-2017
2         Feb-2018
3         Aug-2016
4         Aug-2016
            ...   
826525    May-2016
826526    Jun-2013
826527    Dec-2013
826528    Aug-2015
826529    Jul-2015
Name: issue_d, Length: 826530, dtype: object

In [ ]:
data.loc[0, 'issue_d'][0:3]

'Feb'

In [ ]:
data.loc[0, 'issue_d'][-4:]

'2015'

In [ ]:
for column in date_columns:
    data[column + '_month'] = data[column].apply(lambda x: x[0:3])
    data[column + '_year'] = data[column].apply(lambda x: x[-4:])

In [ ]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,disbursement_method,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year
0,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,10+ years,MORTGAGE,120000.0,...,Cash,N,Feb,2015,Oct,1981,Mar,2019,Mar,2019
1,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,10+ years,RENT,52000.0,...,Cash,N,Dec,2017,Jun,2005,Oct,2018,Oct,2018
2,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,10+ years,OWN,65000.0,...,Cash,N,Feb,2018,Jul,1999,Mar,2019,Mar,2019
3,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,10+ years,MORTGAGE,100000.0,...,Cash,N,Aug,2016,Aug,1990,Sep,2017,Mar,2019
4,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,5 years,RENT,120000.0,...,Cash,N,Aug,2016,Feb,2007,Jul,2018,Mar,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,10+ years,MORTGAGE,70000.0,...,Cash,N,May,2016,Mar,2003,Mar,2019,Mar,2019
826526,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,10+ years,MORTGAGE,98000.0,...,Cash,N,Jun,2013,Aug,2002,Sep,2013,Sep,2013
826527,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,7 years,MORTGAGE,49000.0,...,Cash,N,Dec,2013,Jun,2007,Aug,2015,Mar,2019
826528,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,7 years,MORTGAGE,37000.0,...,Cash,N,Aug,2015,Jan,2004,Aug,2018,Nov,2018


In [ ]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,disbursement_method,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year
0,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,10+ years,MORTGAGE,120000.0,...,Cash,N,Feb,2015,Oct,1981,Mar,2019,Mar,2019
1,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,10+ years,RENT,52000.0,...,Cash,N,Dec,2017,Jun,2005,Oct,2018,Oct,2018
2,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,10+ years,OWN,65000.0,...,Cash,N,Feb,2018,Jul,1999,Mar,2019,Mar,2019
3,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,10+ years,MORTGAGE,100000.0,...,Cash,N,Aug,2016,Aug,1990,Sep,2017,Mar,2019
4,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,5 years,RENT,120000.0,...,Cash,N,Aug,2016,Feb,2007,Jul,2018,Mar,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,10+ years,MORTGAGE,70000.0,...,Cash,N,May,2016,Mar,2003,Mar,2019,Mar,2019
826526,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,10+ years,MORTGAGE,98000.0,...,Cash,N,Jun,2013,Aug,2002,Sep,2013,Sep,2013
826527,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,7 years,MORTGAGE,49000.0,...,Cash,N,Dec,2013,Jun,2007,Aug,2015,Mar,2019
826528,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,7 years,MORTGAGE,37000.0,...,Cash,N,Aug,2015,Jan,2004,Aug,2018,Nov,2018


In [ ]:
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
for column in date_columns:
    data[column + '_month'] = data[column + '_month'].apply(lambda x: month_ordering.index(x))

In [ ]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,disbursement_method,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year
0,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,10+ years,MORTGAGE,120000.0,...,Cash,N,1,2015,9,1981,2,2019,2,2019
1,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,10+ years,RENT,52000.0,...,Cash,N,11,2017,5,2005,9,2018,9,2018
2,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,10+ years,OWN,65000.0,...,Cash,N,1,2018,6,1999,2,2019,2,2019
3,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,10+ years,MORTGAGE,100000.0,...,Cash,N,7,2016,7,1990,8,2017,2,2019
4,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,5 years,RENT,120000.0,...,Cash,N,7,2016,1,2007,6,2018,2,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,10+ years,MORTGAGE,70000.0,...,Cash,N,4,2016,2,2003,2,2019,2,2019
826526,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,10+ years,MORTGAGE,98000.0,...,Cash,N,5,2013,7,2002,8,2013,8,2013
826527,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,7 years,MORTGAGE,49000.0,...,Cash,N,11,2013,5,2007,7,2015,2,2019
826528,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,7 years,MORTGAGE,37000.0,...,Cash,N,7,2015,0,2004,7,2018,10,2018


In [ ]:
for column in data.columns:
    try:
        data[column] = data[column].astype(np.float)
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,disbursement_method,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year
0,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,10+ years,MORTGAGE,120000.0,...,Cash,N,1.0,2015.0,9.0,1981.0,2.0,2019.0,2.0,2019.0
1,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,10+ years,RENT,52000.0,...,Cash,N,11.0,2017.0,5.0,2005.0,9.0,2018.0,9.0,2018.0
2,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,10+ years,OWN,65000.0,...,Cash,N,1.0,2018.0,6.0,1999.0,2.0,2019.0,2.0,2019.0
3,20000.0,20000.0,20000.0,36 months,9.49,640.57,B,10+ years,MORTGAGE,100000.0,...,Cash,N,7.0,2016.0,7.0,1990.0,8.0,2017.0,2.0,2019.0
4,24575.0,24575.0,24575.0,60 months,14.49,578.08,C,5 years,RENT,120000.0,...,Cash,N,7.0,2016.0,1.0,2007.0,6.0,2018.0,2.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826525,35000.0,35000.0,35000.0,36 months,15.31,1218.61,C,10+ years,MORTGAGE,70000.0,...,Cash,N,4.0,2016.0,2.0,2003.0,2.0,2019.0,2.0,2019.0
826526,19750.0,19750.0,19750.0,36 months,17.27,706.80,C,10+ years,MORTGAGE,98000.0,...,Cash,N,5.0,2013.0,7.0,2002.0,8.0,2013.0,8.0,2013.0
826527,11325.0,11325.0,11325.0,60 months,18.55,290.98,D,7 years,MORTGAGE,49000.0,...,Cash,N,11.0,2013.0,5.0,2007.0,7.0,2015.0,2.0,2019.0
826528,5450.0,5450.0,5450.0,36 months,14.65,188.00,C,7 years,MORTGAGE,37000.0,...,Cash,N,7.0,2015.0,0.0,2004.0,7.0,2018.0,10.0,2018.0


In [ ]:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column] == 'object'}

{'term': [' 60 months', ' 36 months'],
 'grade': ['B', 'E', 'C', 'A', 'D', 'F', 'G'],
 'emp_length': ['10+ years',
  '5 years',
  '7 years',
  '2 years',
  '1 year',
  '< 1 year',
  '4 years',
  '6 years',
  '3 years',
  '9 years',
  '8 years'],
 'home_ownership': ['MORTGAGE', 'RENT', 'OWN', 'ANY', 'NONE', 'OTHER'],
 'verification_status': ['Verified', 'Not Verified', 'Source Verified'],
 'issue_d': ['Feb-2015',
  'Dec-2017',
  'Feb-2018',
  'Aug-2016',
  'Oct-2016',
  'Jan-2018',
  'Jan-2015',
  'Sep-2014',
  'May-2015',
  'Apr-2016',
  'Sep-2016',
  'Oct-2017',
  'Jun-2015',
  'Jun-2013',
  'Oct-2013',
  'Jul-2017',
  'May-2018',
  'Jan-2013',
  'Jan-2017',
  'Nov-2018',
  'Jul-2014',
  'Sep-2018',
  'Aug-2017',
  'Oct-2015',
  'Jul-2016',
  'Aug-2018',
  'Aug-2015',
  'Sep-2015',
  'Feb-2017',
  'Mar-2017',
  'Mar-2018',
  'Nov-2014',
  'Mar-2013',
  'Oct-2014',
  'Apr-2015',
  'Feb-2014',
  'Nov-2017',
  'Apr-2013',
  'May-2013',
  'Mar-2014',
  'Jul-2018',
  'Dec-2016',
  'Jul-201

In [ ]:
target = 'grade'


binary_features = ['term', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']
binary_positives = [' 60 months', 'y', 'w', 'Individual', 'Y', 'Cash', 'Y']

ordinal_features = ['emp_length']
emp_ordering = [
    '< 1 year',
    '1 year',
    '2 years',
    '3 years',
    '4 years',
    '5 years',
    '6 years',
    '7 years',
    '8 years',
    '9 years',
    '10+ years'
]

nominal_features = ['home_ownership', 'verification_status', 'loan_status', 'purpose', 'addr_state']

In [ ]:
# Encoding functions

def binary_encode(df, column, positive_value):
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)

def ordinal_encode(df, column, ordering):
    df[column] = df[column].apply(lambda x: ordering.index(x))

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new 

In [ ]:
# Perform encoding

for feature, positive_value in zip(binary_features, binary_positives):
    binary_encode(data, feature, positive_value)

ordinal_encode(data, 'emp_length', emp_ordering)

for feature in nominal_features:
    data = onehot_encode(data, feature)

In [ ]:
(data.dtypes == 'object').sum()

5

In [ ]:
data[target].value_counts()

C    242612
B    240025
A    151679
D    120849
E     50987
F     15711
G      4667
Name: grade, dtype: int64

In [ ]:
# Encoding label column

label_encoder = LabelEncoder()

data[target] = label_encoder.fit_transform(data[target])

target_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}
target_mappings

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

In [ ]:
data = data.drop(issue_d, axis=1)

NameError: ignored

In [ ]:
y = data['grade']
X = data.drop('grade', axis=1)

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)